#### Contents

0. [Load data and preprocess](#Load-data-and-preprocess)
1. [Initialize VRAE object](#Initialize-VRAE-object)
2. [Fit the model onto dataset](#Fit-the-model-onto-dataset)
3. [Transform the input timeseries to encoded latent vectors](#Transform-the-input-timeseries-to-encoded-latent-vectors)
4. [Save the model to be fetched later](#Save-the-model-to-be-fetched-later)

### Import required modules

In [17]:
from model.vrae import VRAE

from model.utils import *
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import trange
import tqdm

### Input parameters

In [18]:
dload = './saved_model' #download directory

### Hyper parameters

### Load data and preprocess
- `folder` : data location
- `cols_to_remove` : generation 수행하지 않을 column 제거

**TODO : 해당 변수에 대한 처리를 어떻게 해줘야하는가 확인 작업이 필요함**

~~~
YYYYMMDD : 년월일
HHMMSS : 시분초
MNG_NO : 장비번호
IF_IDX : 회선 index
~~~

- 현재는 분석의 편의를 위해 ['YYYYMMDD', 'HHMMSS']만 제거해줌

In [ ]:
# params
folder = 'data'
cols_to_remove = ['YYYYMMDD', 'HHMMSS']

# load data
df_total = load_data(folder, cols_to_remove)

# shape
print(df_total.shape)

In [ ]:
class HamonDataset(Dataset):
    def __init__(self, data, window):
        self.data = torch.Tensor(data)
        self.window = window
 
    def __len__(self):
        df_len = len(self.data)
        left = df_len%window
#         return len(self.data)-left
        return len(self.data) // self.window -1
    
    def __getitem__(self, index):
#         x = self.data[index*self.window:index*(self.window+1)]
        x = self.data[index*self.window:(index+1)*(self.window)]
        return x

In [ ]:
data = df_total
window = 100

In [ ]:
train_dataset = HamonDataset(data, window)
train_dataset

In [ ]:
train_dataset[0].shape

**Fetch `sequence_length` from dataset**

In [ ]:
sequence_length = train_dataset[0].shape[0]
sequence_length

**Fetch `number_of_features` from dataset**

This config corresponds to number of input features

In [ ]:
number_of_features = train_dataset[0].shape[1]
number_of_features

### Parameters

In [ ]:
n_epochs = 50
hidden_size = 90
hidden_layer_depth = 1
latent_length = 20
batch_size = 32
learning_rate = 0.0005
dropout_rate = 0.2
optimizer = 'Adam' # options: ADAM, SGD
cuda = True # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
loss = 'MSELoss' # options: SmoothL1Loss, MSELoss
block = 'LSTM' # options: LSTM, GRU

### Initialize VRAE object

VRAE inherits from `sklearn.base.BaseEstimator` and overrides `fit`, `transform` and `fit_transform` functions, similar to sklearn modules

In [ ]:
vrae = VRAE(sequence_length=sequence_length,
            number_of_features = number_of_features,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate,
            optimizer = optimizer, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            loss = loss,
            block = block,
            dload = dload)

### Fit the model onto dataset

In [ ]:
vrae.fit(train_dataset)

#If the model has to be saved, with the learnt parameters use:
# vrae.fit(dataset, save = True)

In [ ]:
vrae.save('vrae.pth')

# To load a presaved model, execute:
# vrae.load('vrae.pth')

### Reconstruct

In [16]:
reconstruction = vrae.transform(train_dataset)
reconstruction

RuntimeError: number of dims don't match in permute

### Save the model to be fetched later